<a href="https://colab.research.google.com/github/torresmateo/penguin-tf-workshop/blob/master/D3_2_Clasificacion_sarcasmo.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Detección de Sarcasmo en Títulos de noticias

Un ejemplo interesante de clasificación de texto es la detección de sarcasmo en títulos de noticias. El [dataset](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection) está disponible en Kaggle, sin embargo este código provee código para descargarlo de forma automática si estás usando Google Colab.

Primero que nada, importamos las bibliotecas necesarias

In [ ]:
%tensorflow_version 2.x
import numpy as np
import tensorflow as tf
# para disminuir la cantidad de código a escribir, importamos 
# el Tokenizer y pad_sequences de forma directa
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
plt.style.use('default')
import gdown
import os
import json

Bajamos los datos y guardamos la ubicación del archivo

In [ ]:

url = 'https://drive.google.com/uc?id=1MF0nvMJyr62S8PwKFfVAVVIxybxSyMa8'
output = '/tmp/sarcasm.json'
gdown.download(url, output, quiet=False)

Cargamos los datos del dataset a una lista de oraciones y sus labels respectivos

In [ ]:
with open("/tmp/sarcasm.json", 'r') as f:
    dataset = json.load(f)

oraciones = []
labels = []

for ejemplo in dataset:
    oraciones.append(ejemplo['headline'])
    labels.append(ejemplo['is_sarcastic'])

exploramos los datos

In [ ]:
print('Cnatidad de ejemplos en el dataset:',len(oraciones))
print(f'Primeros ejemplos del dataset:')
for i in range(5):
    print(f'\n\t{oraciones[i]} \n\t\tEs sarcasmo? {labels[i]}\n')

# Partir el dataset

Distribuimos el dataset en testing y training. Como vimos arriva, hay 28619 ejemplos. Podemos usar 20000 ejemplos para training (cerca del 70%), y los restantes 8619 para testing.  

In [ ]:
particion = 20000

oraciones_train = oraciones[0:particion]
oraciones_test = oraciones[particion:]
labels_train = np.array(labels[0:particion])
labels_test = np.array(labels[particion:])

# Codificar el texto. 

Ahora usamos el tokenizer para codificar el texto a una forma numérica. 

Siempre se debe tener en cuenta que se asume que no tenemos acceso a los datos de testing, por lo cual es importante definir el *out of value token* y ejecutar `fit_on_texts` solo en el *training set*.

In [ ]:
# establecemos el límite de palabras que se usan a 10000 
# (un número razonable, que debería contener a las palabras
#  más comunes del inglés.)
tokenizer = Tokenizer(num_words=10000, oov_token='<???>')

# aprendemos el vocabulario
tokenizer.fit_on_texts(oraciones_train)
vocabulario = tokenizer.word_index

# transformamos el training set a secuencias
sec_train = tokenizer.texts_to_sequences(oraciones_train)

# hacemos el padding de las secuencias 
# (100 debería bastar para la mayoría de los titulares)
# indicamos que se ignoren las palabras posteriores, y que los ceros
# se agreguen al final de las secuencias.
pad_train = pad_sequences(sec_train, maxlen=100, padding='post', truncating='post')

# procesamos el testing set de manera similar
sec_test = tokenizer.texts_to_sequences(oraciones_test)
pad_test = pad_sequences(sec_test, maxlen=100, padding='post', truncating='post')


# Creamos la red neuronal

Creamos y compilamos nuestro clasificador

Aquí, se introducen dos nuevos tipos de *layer*: 

1. *Embedding* layer, que construirá vectores a partir de nuestros ejemplos. Si leemos la [documentación](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) vemos dos parámetros importantes:
  * `input_dim`: el tamaño del vocabulario
  * `output_dim`: la dimensión del vector creado por el layer.

2. [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D), funciona de forma similar a los demás tipos de pooling. Vea el resumen del modelo para entender el efecto de este layer sobre los datos

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=100),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

para entender un poco mejor el modelo con los nuevos layers, podemos ver un resumen

In [ ]:
model.summary()

Entrenamos el modelo como siempre.

In [ ]:
historia = model.fit(pad_train, labels_train, epochs=30, validation_data=(pad_test, labels_test), verbose=2)

copio la función de la clase pasada para ver la evolución de nuestros resultados.

In [ ]:
def ver_historia(historia, titulo = '', ax = None):
    """
    Visualizar una la historia de un modelo, 
    se hará una figura que muestre la evolución de la
    función de costo y de la precisión del modelo con
    respecto a los epochs.

    Parameters
    ----------
    historia : keras History
        Es lo que retorna la llamada a `model.fit`
    titulo : str
        el título del ax de arriba
    ax : np.array
        si se provee, no se creará una imagen nueva y se usará
        `ax` en su lugar. Se debe proveer 2 ejes de una figura
        de pyplot.
    """
    create = ax is None
    if create:
        fig, ax = plt.subplots(2,1,figsize=(10,8), dpi=100)
    acc      = historia.history['accuracy']
    val_acc  = historia.history['val_accuracy']
    loss     = historia.history['loss']
    val_loss = historia.history['val_loss']
    epochs = range(len(acc))
    ax[0].grid(True)
    ax[0].plot(epochs, acc, label=f"Entrenamiento - {titulo}")
    ax[0].plot(epochs, val_acc, label=f"Evaluación - {titulo}")
    ax[0].set_ylabel('Precisión')
    ax[0].set_xlabel('Epoch')
    #ax[0].set_ylim(0, 1.1)
    ax[0].legend()
    ax[1].grid(True)
    ax[1].plot(epochs, loss, label=f"Entrenamiento - {titulo}")
    ax[1].plot(epochs, val_loss, label=f"Evaluación - {titulo}")
    ax[1].set_ylabel('Costo')
    ax[1].set_xlabel('Epoch')
    ax[1].legend()
    if create:
        plt.show()

In [ ]:
ver_historia(historia)

Vemos que nuestro modelo no es muy bueno para generalizar las predicciones en este dataset. tenemos más o menos 80% de precisión luego de entrenar por 30 *epochs*. 

# TAREA (para la casa)

Este modelo es capaz de obtener mucho mejor precisión que lo que vimos luego de los 30 *epochs*. El ejercicio consiste en modificar los parámetros de nuestro modelo y nuestro tokenizador:

* la cantidad de palabras en el vocabulario
* la longitud máxima del *padding*
* las dimensiones del *embedding*
* etc.

Modifique estos valores y vaya explorando diferentes combinaciones para lograr una precisión del 90%

# Créditos 

Este notebook utiliza y modifica contenido del curso online [TensorFlow in Practice](https://www.deeplearning.ai/tensorflow-in-practice/).